In [1]:
import sys
sys.path.append("/mnt/md0/virtualenv/snowpark/lib/python3.13.5/site-packages")
from snowflake.snowpark.session import Session

C:\Users\Darker Blade\AppData\Local\Temp\ipykernel_12188\1632791329.py:3: DeprecationWarning: This package has been renamed to snowflake_uuid and will be removed shortly. Please update immediately.
  from snowflake.snowpark.session import Session


ModuleNotFoundError: No module named 'snowflake.snowpark'; 'snowflake' is not a package

In [ ]:
from snowflake.snowpark import Session

connection_parameters = {
    "account": "oqjwzln.eu-west-3",       # Lowercase full Snowflake account locator + region
    "user": "rts_user_ext",                  # Your Snowflake username (case-sensitive)
    "password": "TempPassword123!",        # Fill this in securely
    "role": "ACCOUNTADMIN",               # Role you are using
    "warehouse": "compute_wh",      # Example: "COMPUTE_WH" or whatever you're assigned
    "database": "rts_data",               # From your current database context
    "schema": "PUBLIC"                    # Schema from the left-side tree
}

session = Session.builder.configs(connection_parameters).create()

# 1.  Reusable file format & table  (run once)  ──────────────────────
session.sql("""
    CREATE OR REPLACE FILE FORMAT csv_ff
      TYPE = 'CSV'
      SKIP_HEADER = 1
      FIELD_OPTIONALLY_ENCLOSED_BY = '"'
""").collect()

session.sql("""
    CREATE OR REPLACE TABLE MY_TABLE (
        col1 STRING,
        col2 STRING,
        …                     -- define explicit types for better control
)
""").collect()

# 2.  Stage & upload the local file  ─────────────────────────────────
session.sql("CREATE OR REPLACE STAGE mystage FILE_FORMAT = csv_ff").collect()

put_results = session.file.put(
        local_file_name="../data/df.csv",
        stage_location="@mystage",
        overwrite=True,
        auto_compress=False      # compress automatically if file is large
)
print(put_results[0].status)     # quick sanity check

# 3.  Load into the table  ───────────────────────────────────────────
session.sql("""
    COPY INTO MY_TABLE
    FROM @mystage/df.csv
    FILE_FORMAT = (FORMAT_NAME = csv_ff)
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    ON_ERROR = 'CONTINUE'
""").collect()

session.close()


In [ ]:
!pip list | grep snowflake

In [ ]:
pip install "snowflake[ml]" -U

In [2]:
pip install snowflake-snowpark-python

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python ==3.8.*; 0.11.0 Requires-Python ==3.8.*; 0.12.0 Requires-Python ==3.8.*; 0.6.0 Requires-Python ==3.8.*; 0.7.0 Requires-Python ==3.8.*; 0.8.0 Requires-Python ==3.8.*; 0.9.0 Requires-Python ==3.8.*; 1.0.0 Requires-Python ==3.8.*; 1.1.0 Requires-Python ==3.8.*; 1.10.0 Requires-Python >=3.8, <3.12; 1.11.0 Requires-Python >=3.8, <3.12; 1.11.1 Requires-Python >=3.8, <3.12; 1.12.0 Requires-Python >=3.8, <3.12; 1.12.1 Requires-Python >=3.8, <3.12; 1.13.0 Requires-Python >=3.8, <3.12; 1.14.0 Requires-Python <3.12,>=3.8; 1.15.0 Requires-Python <3.12,>=3.8; 1.16.0 Requires-Python <3.12,>=3.8; 1.17.0 Requires-Python <3.12,>=3.8; 1.18.0 Requires-Python <3.12,>=3.8; 1.19.0 Requires-Python <3.12,>=3.8; 1.2.0 Requires-Python ==3.8.*; 1.20.0 Requires-Python <3.12,>=3.8; 1.21.0 Requires-Python <3.12,>=3.8; 1.21.1 Requires-Python <3.12,>=3.8; 1.22.0 Requires-Python <3.12,>=3.8; 1.22.1 Requires-Python <3.